In [1]:
!pip install tabula-py
!pip install pdf2image
!pip install tensorflow==2.0.0
!pip uninstall h5py
!pip install 'h5py==2.10.0'
!apt-get install poppler-utils
!pip install PyPDF2

     |████████████████████████████████| 12.0 MB 95 kB/s 
     |████████████████████████████████| 86.3 MB 20 kB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 449 kB 36.2 MB/s 
     |████████████████████████████████| 3.8 MB 39.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=a2f8a148e4ea498a1a6615508558e6e104cba1fdd8fd73eaac3d8a444c7abe0e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installati

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from pdf2image import convert_from_path


# Store Pdf with convert_from_path function
for pdf in glob.glob('C:\ASSET/PDF/*.pdf'):
    images = convert_from_path(pdf,poppler_path = r'C:\ASSET\poppler-21.09.0\Library\bin')
    #print(images)
    images[0].save(str(pdf)[:-4]+'.jpg', 'JPEG')
	

# XML to CSV Conversion
def xml_to_csv(path):
xml_list = []
for xml_file in glob.glob(path+'/*.xml'):
tree = ET.parse(xml_file)
root = tree.getroot()
for member in root.findall('object'):
value = (root.find('filename').text,int(member[4][0].text),int(member[4][1].text),int(member[4][2].text),int(member[4][3].text),member[0].text)
xml_list.append(value)
        column_name = ['filename','xmin','ymin','xmax','ymax','class']
        xml_df = pd.DataFrame(xml_list,columns = column_name)
    return xml_df
   
def main():
    image_path = os.path.join(os.getcwd(),'C:\ASSET\')
    print(image_path)
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('new199.csv',index=None)
    print('successfully converted xml to csv.')

In [ ]:
#Training the model using the bounding box coordinates
import tensorflow as tf
#from src import config
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten,Dense,Dropout,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import pickle
from cv2 import cv2
import os

#load the contents of annotated csv file
print("[INFO] loading dataset ..")
data = []
labels = []
targets = []
imagePaths = []

#for csvPath in paths.list_files(config.ANNOTS_PATH,validExts=(".csv")):
    #rows = open(csvPath).read().strip().split("\n")

rows = open('new.csv').read().strip().split("\n")

k=0
for row in rows:
    # break the row into the filename, bounding box coordinates,
    # and class label
    row = row.split(",")
    (filename, startX, startY, endX, endY, label) = row
    # derive the path to the input image, load the image (in OpenCV
    # format), and grab its dimensions
    #    imagepath = os.path.sep.join([config.IMAGES_PATH,label,filename])
    imagepath = "C:\\ASSET\\PDF\\" + filename
    print("imagepath",imagepath)
    image = cv2.imread(imagepath)
    k = k+1
    print("FOUND AND READ IMAGE: ",imagepath,"with k:",k)
    crop_image = image[int(startY): int(endY),int(startX): int(endX)]
    
    (h, w) = image.shape[:2]
    # scale the bounding box coordinates relative to the spatial dimensions of the input image
    startX = float(startX) / w
    startY = float(startY) / h
    endX = float(endX) / w
    endY = float(endY) / h
       
    # load the image and preprocess it
    image = load_img(imagepath, target_size=(224, 224))
    image = img_to_array(image)
    # update our list of data, class labels, bounding boxes, and
    # image paths
    data.append(image)
    labels.append(label)
    targets.append((startX, startY, endX, endY))
    imagePaths.append(imagepath)
       
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
targets = np.array(targets, dtype="float32")
imagePaths = np.array(imagePaths)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well
if len(lb.classes_) == 2:
    labels = to_categorical(labels)
    
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(data, labels, targets, imagePaths,test_size=0.20, random_state=42)

# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]
# write the testing image paths to disk so that we can use then
# when evaluating/testing our object detector
print("[INFO] saving testing image paths...")
TEST_PATHS = "C:\\ASSET\\PDF\\test_images.txt"
f = open(TEST_PATHS, "w")
f.write("\n".join(testPaths))
f.close()

# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(256, activation="relu")(flatten)
bboxHead = Dropout(0.2)(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dropout(0.2)(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dropout(0.2)(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dropout(0.2)(bboxHead)
bboxHead = Dense(4, activation="sigmoid",name="bounding_box")(bboxHead)
# construct a second fully-connected layer head, this one to predict the class label
#softmaxHead = Dense(512, activation="relu")(flatten)
#softmaxHead = Dropout(0.5)(softmaxHead)
#softmaxHead = Dense(512, activation="relu")(softmaxHead)
#softmaxHead = Dropout(0.5)(softmaxHead)
#softmaxHead = Dense(len(lb.classes_), activation="softmax",name="class_label")(softmaxHead)

# construct the model we will fine tune for bbox regression
model = Model(inputs=vgg.input,outputs=(bboxHead))

losses = {"bounding_box":"mean_squared_error"}

lossWeights = {"bounding_box": 1.0}

NUM_EPOCHS = 100
INIT_LR = 1e-4
BATCH_SIZE = 128

# initialize the optimizer, compile the model, and show the model summary
opt = Adam(lr=INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

# construct a dictionary for our target training outputs
trainTargets = {
    "bounding_box": trainBBoxes
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
    "bounding_box": testBBoxes
}

# train the network for bounding box regression
print("[INFO] training bounding box regression...")
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.save_weights(checkpoint_path.format(epoch=0))

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=25*BATCH_SIZE)
    
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
       callback= [cp_callback],
    verbose=1)
# serialize the model to disk
print("[INFO] saving object detector model...")
TABLE_MODEL_PATH = os.path.sep.join([os.getcwd(),"Table_Detector.h5"])
TABLE_PICKLE = os.path.sep.join([os.getcwd(),"Table_lb.pickle"])
model.save(TABLE_MODEL_PATH)
# serialize the label binarizer to disk
f = open(TABLE_PICKLE, "wb")
f.write(pickle.dumps(lb))
f.close()

PLOT_PATH = "C:\\ASSET\\PDF\\"
#BASE_OUTPUT = os.getcwd()
BASE_OUTPUT = "C:\\ASSET\\PDF\\"
# plot the total loss, label loss, and bounding box loss
lossNames = ["loss"]
N = np.arange(0,NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))
# loop over the loss names
for (i, l) in enumerate(lossNames):
    # plot the loss for both the training and validation data
    title = "Loss for {}".format(l) if l != "loss" else "Total loss"
    ax[i].set_title(title)
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("Loss")
    ax[i].plot(N, H.history[l], label=l)
    ax[i].plot(N, H.history["val_" + l], label="val_" + l)
    ax[i].legend()
# save the losses figure and create a new figure for the accuracies
plt.tight_layout()
plotPath = os.path.sep.join([BASE_OUTPUT, "losses.png"])
plt.savefig(plotPath)
plt.close()

N = NUM_EPOCHS

plotPath = os.path.sep.join([BASE_OUTPUT, "acc.png"])
plt.savefig(plotPath)

plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding box regression loss on training set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)

In [44]:
#Test the model
from pdf2image import convert_from_path
fn = 'test1.pdf'
images = convert_from_path(fn)
images[0].save('test.png', 'PNG')
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
import io
from PyPDF2 import PdfFileReader 
from tensorflow.keras.models import load_model
from tabula import read_pdf
import math
from cv2 import cv2
with io.open(fn,mode = "rb") as f:
  input_pdf = PdfFileReader(f)
  media_box = input_pdf.getPage(0).mediaBox
min_pt = media_box.lowerLeft
max_pt = media_box.upperRight
image = load_img('test.png',target_size = (224,224))
image = img_to_array(image)/255.0
image = np.expand_dims(image,axis = 0)

borderless_model = load_model("/content/drive/MyDrive/ts/Table_Detector_borderless.h5",compile=False)
borderless_preds = borderless_model.predict(image)[0]

(borderless_startX,borderless_startY,borderless_endX,borderless_endY) = borderless_preds
from cv2 import cv2
image = cv2.imread('test.png')

image_cp = image.copy()

(h,w) = image.shape[:2]

borderless_startX = int(borderless_startX*w)
borderless_startY = int(borderless_startY*h)
borderless_endX = int(borderless_endX*w)
borderless_endY = int(borderless_endY*h)

#final_pdf_coordinates = str(math.floor(borderless_startY/2)-5)+","+str(math.floor(min_pt[0]))+"," + str(math.floor(borderless_endY/2)) + ","+ str(math.ceil(max_pt[0]))
final_pdf_coordinates = str(math.floor(borderless_startY/2)-20)+","+str(math.floor(borderless_startX/2))+"," + str(math.floor(borderless_endY/2)+60) + ","+ str(math.floor(borderless_endX/2)-5)


print("FINAL_TABLE_COORDINATES: ",final_pdf_coordinates)

#cv2.rectangle(image,(int(min_pt[0]),borderless_startY,),(int(max_pt[0]*2),borderless_endY),(0,255,0),2)
#cv2.rectangle(image,(borderless_startX,borderless_startY-5,),(borderless_endX,borderless_endY),(0,255,0),2)
cv2.rectangle(image,(borderless_startX,borderless_startY,),(borderless_endX,borderless_endY),(0,255,0),2)

imS = cv2.resize(image,(560,640))

tables = read_pdf(fn,area = [final_pdf_coordinates],stream = True,output_format = "dataframe",pandas_options = {'header':None})
if len(tables) > 0:
	df = pd.DataFrame(np.concatenate(tables))
	
else:
  print("No table")
  df = pd.DataFrame()

cv2.imwrite('op.jpg', imS)
df.head()
#cv2.imshow("model_output",imS)

#cv2.waitkey(0)

'pages' argument isn't specified.Will extract only from page 1 by default.


FINAL_TABLE_COORDINATES:  70,4,150,838


,0,1,2,3,4,5,6
0,Category,Projected_Cost,Actual_Cost,Difference,Projected_Cost,Actual_Cost,Difference
1,Home,100,50,50,100,50,50
2,Health,100,50,50,100,50,50
3,Life,100,50,50,100,50,50
4,Other,100,50,50,100,50,50


In [39]:

fn = 'test1.pdf'
tables = read_pdf(fn,area = [70,4,150,838],stream = True,output_format = "dataframe",pandas_options = {'header':None}, pages=1)
df = pd.DataFrame(np.concatenate(tables))
df

,0,1,2,3,4,5,6
0,Category,Projected_Cost,Actual_Cost,Difference,Projected_Cost,Actual_Cost,Difference
1,Home,100,50,50,100,50,50
2,Health,100,50,50,100,50,50
3,Life,100,50,50,100,50,50
4,Other,100,50,50,100,50,50


# New Section

In [ ]:
#!pip install 'h5py==2.10.0' --force-reinstall
#!pip uninstall h5py
!pip install 'h5py==2.10.0'
#!pwd
#/content/test.jpg
#/content/sample_data/Table_Detector.h5

  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


In [ ]:
import pandas as pd
import numpy as np
from tabula import read_pdf
tables = read_pdf('test.pdf',area = [338,200,608,596],stream = True,output_format = "dataframe",pandas_options = {'header':None},pages =1)
print(tables)
# df_concat = pd.DataFrame(np.concatenate(tables))
# df_concat.head()

The output file is empty.


[]


In [ ]:
!pip install tabula-py
!pip install pdf2image
!pip install tensorflow=2.0.0
!pip uninstall h5py
!pip install 'h5py==2.10.0'
!apt-get install poppler-utils
!pip install PyPDF2

     |████████████████████████████████| 77 kB 2.8 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=2289b89391bfbbefe0a2ed7bdce780260e0f7b6c4150a898ee3d02b7025310ea
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [ ]:
# tables = read_pdf('test.pdf',stream = True,output_format = "dataframe",pandas_options = {'header':None})
# if len(tables) > 0:
# 	df = pd.DataFrame(np.concatenate(tables))
# df.head()
#df_concat,currency_code,total_amount = dataframe_processing(df)
!pwd

/content


# New Section